In [ ]:
!pip install -qU langchain-dappier langchain langchain-openai langchain-community langchain-core openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.0 MB/s eta 0:00:00


In [ ]:
import os
from getpass import getpass

os.environ["DAPPIER_API_KEY"] = getpass("Enter your Dappier API key: ")

os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API key: ")

Enter your Dappier API key: ··········
Enter your OpenAI API key: ··········


In [ ]:
from langchain_dappier import DappierRealTimeSearchTool

search_tool = DappierRealTimeSearchTool()
print("Real-time search tool ready:", search_tool)

Real-time search tool ready: 


In [ ]:
from langchain_dappier import DappierAIRecommendationTool

recommendation_tool = DappierAIRecommendationTool(
    data_model_id="dm_01j0pb465keqmatq9k83dthx34",
    similarity_top_k=3,
    ref="sportsnaut.com",
    num_articles_ref=2,
    search_algorithm="most_recent",
)
print("Recommendation tool ready:", recommendation_tool)

Recommendation tool ready: similarity_top_k=3 ref='sportsnaut.com' num_articles_ref=2


In [ ]:
from langchain.chat_models import init_chat_model

llm = init_chat_model(
    model="gpt-3.5-turbo",
    model_provider="openai",
    temperature=0,
)
llm_with_tools = llm.bind_tools([search_tool])
print("✅ llm_with_tools ready")

✅ llm_with_tools ready


In [ ]:
import datetime
from langchain_core.prompts import ChatPromptTemplate

today = datetime.datetime.today().strftime("%Y-%m-%d")
prompt = ChatPromptTemplate([
    ("system", f"You are a helpful assistant. Today is {today}."),
    ("human", "{user_input}"),
    ("placeholder", "{messages}"),
])

llm_chain = prompt | llm_with_tools
print("✅ llm_chain built")

✅ llm_chain built


In [ ]:
from langchain_core.runnables import RunnableConfig, chain

@chain
def tool_chain(user_input: str, config: RunnableConfig):
    ai_msg = llm_chain.invoke({"user_input": user_input}, config=config)
    tool_msgs = search_tool.batch(ai_msg.tool_calls, config=config)
    return llm_chain.invoke(
        {"user_input": user_input, "messages": [ai_msg, *tool_msgs]},
        config=config
    )

print("✅ tool_chain defined")

✅ tool_chain defined


In [ ]:
res = search_tool.invoke({"query": "What happened at the last Wrestlemania"})
print("🔍 Search:", res)

🔍 Search: At WrestleMania 41, some major highlights included:

- **John Cena** made history by winning his 17th WWE Championship, marking a huge milestone in his career! 🎉
- In the main event of Night 1, **Seth Rollins** defeated **CM Punk** and **Roman Reigns** in a thrilling triple threat match. 🤼‍♂️
- On Night 2, **Roman Reigns** defended his title against **Cody Rhodes** under Bloodline Rules, and in a big twist, he defeated **Brock Lesnar** to unify both the Universal and WWE Championships! 🏆

The event was packed with excitement and drama, making it a memorable WrestleMania! 🥳


In [ ]:
rec = recommendation_tool.invoke({"query": "latest sports news"})
print("📄 Recommendation:", rec)

out = tool_chain.invoke("Who won the last Nobel Prize?")
print("🤖 Chain output:", out)